
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
# compute a simple cross-tabulation of the counts for the counts of whether or not a resume received a call, 
# based on the perceived race of the applicant
pd.crosstab(index=data['race'], columns=data['call'])


call,0.0,1.0
race,,
b,2278,157
w,2200,235


In [6]:
# create a subset of the data with the only the columns of interest.
calls = data[['race', 'call']]

###Introduction

To perform a statistical analysis on whether race has a significant impact on the rate of callbacks for resumes, we will use the difference between two proportions. The first proportion is the proportion of resumes with black-sounding names that received a call back, and the second proportion is for resumes with white-sounding names that received a call back. 

The null hypothesis (H_0) is that there is no difference in the two proportions. The alternative hypothesis (H_A) is that there is difference in the proportion of resumes that received callbacks, based on race.

####Conditions
The normal model will be applied to this problem. Two conditions must be verified before applying this model to the difference of two proportions:
1. Each proportion separately follows a normal model.
2. The two samples are independant of each other.

To evaluate these conditions, we must first calculate the **pooled proportion**. The pooled proportion is the best-estimate of the callback rate, if the null hypothesis is true.

In [7]:
# pooled proportion is the total number of callbacks, divided by the total number of applicants
n = len(calls)
pooled = sum(calls.call) / n
pooled

0.080492813141683772

Since the data is from a randomized field experiment involving nearly 5,000 résumés sent in response to over 1,300 newspaper ads, the callback results can be treated as independant. To evaluate whether the success-failure condition for the normal model is met, the number of success (callbacks) and failures (no callback) from each group must be at least 10, based on the pooled proportion.

In [8]:
# determine the number of black and white applicants
n_black = len(calls[calls.race=='b'])
n_white = len(calls[calls.race=='w'])

print "Callbacks, Black", n_black * pooled
print "No Callbacks, Black", n_black * (1 - pooled)
print "Callbacks, White", n_white * pooled
print "No Callbacks, White", n_white * (1 - pooled)


Callbacks, Black 196.0
No Callbacks, Black 2239.0
Callbacks, White 196.0
No Callbacks, White 2239.0


The **success-failure condition is satisfied** (all values greater than 10), so the normal model applies.

### Hypothesis Testing
To test the hypothesis, first calculate a point estimate of the difference in callback rates between the two races. Then calculate the standard error using the pooled proportion.

In [9]:
# point estimate of difference in callback rates
callbacks_white = sum(calls[calls.race=='w'].call)
callbacks_black = sum(calls[calls.race=='b'].call)

p_white = callbacks_white / n_white
p_black = callbacks_black / n_black

p_diff = p_white - p_black
p_diff

0.032032854209445585

In [10]:
# Std Error calculated using the pooled proportion
SE = ( (pooled*(1-pooled) / n_white) + (pooled*(1-pooled) / n_black) ) ** 0.5
SE

0.0077968940361704568

Next, compute the test statistic (z) based on the point estimate and std error calculated above. The null value is zero, since the null hypothesis assumes no difference in the two proportions:

In [11]:
p_null = 0
z = (p_diff - p_null) / SE
z

4.1084121524343464

In [12]:
p = stats.norm.sf(abs(z))*2
p

3.9838868375850767e-05

Since the **p-value is much smaller than the significance level (0.05)**, there is convincing evidence to **reject the null hypothesis**. Therefore, there was a statistically significant difference in the proportions of resumes that received callbacks between the two races examined in this study.

### Confidence Intervals

We can also calculate confidence intervals for the difference in proportion of callbacks between the black and white applicants. First, calculate a standard error, based on the sample proportions: 

In [13]:
SE_2 = ( (p_white*(1-p_white) / n_white) + (p_black*(1-p_black) / n_black) ) ** 0.5
SE_2

0.0077833705866767544

For a 95% confidence interval, use a z* = 1.96. The margin of error (ME) and confidence interval (CI) are:

In [15]:
ME = 1.96 * SE_2
print 'Margin of Error:', ME

CI = (p_diff - ME, p_diff + ME)
print 'Confidence Interval:', CI

Margin of Error: 0.0152554063499
Confidence Interval: (0.016777447859559147, 0.047288260559332024)


We are 95% certain that the callback rate for job applications for sales, administrative, and clerical jobs in Boston and Chicago was 1.6 to 4.7 percent higher for resumes containing white-sounding names, compared to those with black-sounding names.